In [8]:
from model import *
from hparams import *
from dataset import get_data_loader, get_data_loader_michigan, get_data_loader_synthesized_michigan

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#new jupyter notebook
from tqdm import tqdm

train_loader = get_data_loader(split='train', args=Hparams.args)
for data in tqdm(train_loader):
    mel_spectrogram, yin, pyin = data#move_data_to_device(data, 'cpu')
    print(mel_spectrogram.shape)
    print(yin.shape)
    print(pyin.shape)
    #assert list(x.shape) == [8, 250, 256]  # shape in [B, T, D],
                                # i.e., [Batch size, num of frame per sample, spectrogram feature dimension]
    #assert list(onset.shape) == list(offset.shape) == list(octave.shape) == list(pitch_class.shape) == [8, 250]
    break
print('Congrats!')

  0%|          | 0/10 [00:09<?, ?it/s]

torch.Size([8, 256, 5])
torch.Size([8, 256])
torch.Size([8, 256])
Congrats!


In [ ]:
#For the Michigan dataset
train_ds, test_ds, data_loader_train, data_loader_test = get_data_loader_michigan(args=Hparams_michigan.args, test_size=0.2)

In [ ]:
from tqdm import tqdm
# to plot
# train_ds.plot_item(0)

# data loading
for data in tqdm(data_loader_train):
    mel_spectrogram_normalised_log_scale_torch, yin_normalised_torch, pyin_normalised_torch, word, toneclass = data
    print(f"(Batch, feature)")
    print(f"Spectrogram: {mel_spectrogram_normalised_log_scale_torch.shape} {type(mel_spectrogram_normalised_log_scale_torch)}")
    print(f"Yin: {yin_normalised_torch.shape} {type(yin_normalised_torch)}")
    print(f"Pyin: {pyin_normalised_torch.shape} {type(pyin_normalised_torch)}")
    print(f"Word: {len(word)} {type(word)}")
    print(f"Toneclass: {toneclass.shape} {type(toneclass)} {toneclass}")
    break
print('Congrats!')

  0%|          | 0/246 [01:05<?, ?it/s]

(Batch, feature)
Spectrogram: torch.Size([32, 128, 75]) <class 'torch.Tensor'>
Yin: torch.Size([32, 481]) <class 'torch.Tensor'>
Pyin: torch.Size([32, 501]) <class 'torch.Tensor'>
Word: 32 <class 'tuple'>
Toneclass: torch.Size([32]) <class 'torch.Tensor'> tensor([1, 1, 4, 2, 4, 1, 2, 3, 1, 2, 2, 4, 3, 2, 3, 4, 3, 1, 4, 1, 2, 1, 4, 2,
        3, 1, 4, 1, 1, 4, 3, 1])
Congrats!


In [ ]:
import torch

from sklearn.metrics import accuracy_score

class Metrics:
    def __init__(self):
        self.buffer = {}

    def update(self, out, tgt, loss):
        with torch.no_grad():
            out = out.argmax(dim=1)
            out = torch.flatten(out)
            tgt = torch.flatten(tgt)

            acc = accuracy_score(tgt.cpu(), out.cpu())

            batch_metric = {
                'loss': loss.item(),
                'accuracy': acc,
            }

            for k in batch_metric:
                if k in self.buffer:
                    self.buffer[k].append(batch_metric[k])
                else:
                    self.buffer[k] = [batch_metric[k]]

    def get_value(self):
        for k in self.buffer:
            self.buffer[k] = sum(self.buffer[k]) / len(self.buffer[k])
        ret = self.buffer
        self.buffer = {}
        return ret

In [ ]:
import os
import time

import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from dataset import get_data_loader, move_data_to_device


def fit(model, args, learning_params):
    # Set paths
    save_model_dir = f"{args['save_model_dir']}{model.feat_dim}_lr-{learning_params['lr']}"
    if not os.path.exists(save_model_dir):
        os.mkdir(save_model_dir)

    model.to(args['device'])
    optimizer = optim.AdamW(model.parameters(), lr=learning_params['lr'])
    loss_func = nn.CrossEntropyLoss()
    metric = Metrics()

    # Start training
    print('Start training...')
    start_time = time.time()
    best_model_id = -1
    min_valid_loss = 10000
    prev_loss = 10000
    threshold = 1e-6

    for epoch in range(1, learning_params['epoch'] + 1):
        model.train()
        
        # Train
        pbar = tqdm(data_loader_train)
        for batch_idx, batch in enumerate(pbar):
            mel_spectrogram_normalised_log_scale_torch, yin_normalised_torch, pyin_normalised_torch, word, tone_class = batch
            tone_class -= 1 # 0-index
        
            x = mel_spectrogram_normalised_log_scale_torch.to(args['device'])
            x = x[:, None, :, :]
            tgt = tone_class.to(args['device'])
            out = model(x)
            loss = loss_func(out, tgt)
            metric.update(out, tgt, loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_description('Epoch {}, Loss: {:.4f}'.format(epoch, loss.item()))
        metric_train = metric.get_value()

        # Validation
        model.eval()
        with torch.no_grad():
            for batch_idx, batch in enumerate(data_loader_test):
                mel_spectrogram_normalised_log_scale_torch, yin_normalised_torch, pyin_normalised_torch, word, tone_class = batch
                tone_class -= 1 # 0-index

                x = mel_spectrogram_normalised_log_scale_torch.to(args['device'])
                x = x[:, None, :, :]
                tgt = tone_class.to(args['device'])
                out = model(x)
                loss = loss_func(out, tgt)
                metric.update(out, tgt, loss)
        metric_test = metric.get_value()

        # Logging
        print('[Epoch {:02d}], Train Loss: {:.5f}, Valid Loss {:.5f}, Time {:.2f}s'.format(
            epoch, metric_train['loss'], metric_test['loss'], time.time() - start_time,
        ))
        print('Split Train Loss, Accuracy: Loss {:.4f} | Accuracy {:.4f}'.format(
            metric_train['loss'],
            metric_train['accuracy']
        ))
        print('Split Test Loss, Accuracy: Loss {:.4f} | Accuracy {:.4f}'.format(
            metric_test['loss'],
            metric_test['accuracy']
        ))

        # Save the best model
        if metric_test['loss'] < min_valid_loss:
            min_valid_loss = metric_test['loss']
            best_model_id = epoch

            save_dict = model.state_dict()
            target_model_path = save_model_dir + '/best_model.pth'
            torch.save(save_dict, target_model_path)

        if abs(metric_test['loss'] - prev_loss) < threshold:
            break

        prev_loss = metric_test['loss']

    print('Training done in {:.1f} minutes.'.format((time.time() - start_time) / 60))
    return best_model_id

In [ ]:
# Set learning params
learning_params = {
    'epoch': 10,
    'lr': 1e-3,
}

model = ToneEval_Base(input_shape=(1, 128, 75), feat_dim=64)
fit(model, args=Hparams.args, learning_params=learning_params)

Start training...


  0%|          | 0/246 [02:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Set learning params
learning_params = {
    'epoch': 10,
    'lr': 1e-3,
}

model = ToneEval_Base(input_shape=(1, 128, 75), feat_dim=512)
fit(model, args=Hparams.args, learning_params=learning_params)

Start training...


Epoch 1, Loss: 0.7652: 100%|██████████| 246/246 [1:25:19<00:00, 20.81s/it]


[Epoch 01], Train Loss: 0.78274, Valid Loss 0.76786, Time 6558.57s
Split Train Loss, Accuracy: Loss 0.7827 | Accuracy 0.9630
Split Test Loss, Accuracy: Loss 0.7679 | Accuracy 0.9768


Epoch 2, Loss: 0.7550: 100%|██████████| 246/246 [1:24:37<00:00, 20.64s/it]


[Epoch 02], Train Loss: 0.75677, Valid Loss 0.75173, Time 12944.62s
Split Train Loss, Accuracy: Loss 0.7568 | Accuracy 0.9874
Split Test Loss, Accuracy: Loss 0.7517 | Accuracy 0.9924


...

Epoch 9, Loss: 0.7437: 100%|██████████| 246/246 [1:24:49<00:00, 20.69s/it]


[Epoch 09], Train Loss: 0.74881, Valid Loss 0.74909, Time 57999.69s
Split Train Loss, Accuracy: Loss 0.7488 | Accuracy 0.9947
Split Test Loss, Accuracy: Loss 0.7491 | Accuracy 0.9950


Epoch 10, Loss: 0.7729: 100%|██████████| 246/246 [1:25:08<00:00, 20.76s/it]


[Epoch 10], Train Loss: 0.75100, Valid Loss 0.74900, Time 64439.55s
Split Train Loss, Accuracy: Loss 0.7510 | Accuracy 0.9929
Split Test Loss, Accuracy: Loss 0.7490 | Accuracy 0.9950
Training done in 1074.0 minutes.


8

In [ ]:
# Set learning params
learning_params = {
    'epoch': 10,
    'lr': 1e-3,
}

model = ToneEval_Base(input_shape=(1, 128, 75))
fit(model, args=Hparams.args, learning_params=learning_params)

Start training...


Epoch 1, Loss: 0.7451: 100%|██████████| 246/246 [1:27:53<00:00, 21.44s/it]


[Epoch 01], Train Loss: 0.78702, Valid Loss 0.75133, Time 6639.65s
Split Train Loss, Accuracy: Loss 0.7870 | Accuracy 0.9564
Split Test Loss, Accuracy: Loss 0.7513 | Accuracy 0.9934


Epoch 2, Loss: 0.7464: 100%|██████████| 246/246 [1:25:02<00:00, 20.74s/it]


[Epoch 02], Train Loss: 0.75874, Valid Loss 0.77568, Time 13062.78s
Split Train Loss, Accuracy: Loss 0.7587 | Accuracy 0.9848
Split Test Loss, Accuracy: Loss 0.7757 | Accuracy 0.9682


...

Epoch 9, Loss: 0.7438: 100%|██████████| 246/246 [1:28:16<00:00, 21.53s/it]


[Epoch 09], Train Loss: 0.75207, Valid Loss 0.75403, Time 59019.99s
Split Train Loss, Accuracy: Loss 0.7521 | Accuracy 0.9917
Split Test Loss, Accuracy: Loss 0.7540 | Accuracy 0.9904


Epoch 10, Loss: 0.7437: 100%|██████████| 246/246 [1:29:48<00:00, 21.91s/it]


[Epoch 10], Train Loss: 0.75009, Valid Loss 0.74684, Time 65815.95s
Split Train Loss, Accuracy: Loss 0.7501 | Accuracy 0.9934
Split Test Loss, Accuracy: Loss 0.7468 | Accuracy 0.9970
Training done in 1096.9 minutes.


8

In [2]:
import os
import time

import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

def fit_segmentation(model, train_data_loader, test_data_loader, args, learning_params):
    # Set paths
    save_model_dir = f"{args['save_model_dir']}{model.feat_dim}_lr-{learning_params['lr']}"
    if not os.path.exists(save_model_dir):
        os.mkdir(save_model_dir)

    model.to(args['device'])
    optimizer = optim.AdamW(model.parameters(), lr=learning_params['lr'])
    loss_func = SegmentLossFunc()
    metric = SegmentMetrics(loss_func)

    # Start training
    print('Start training...')
    start_time = time.time()
    best_model_id = -1
    min_valid_loss = 10000
    prev_loss = 10000
    threshold = 1e-6

    for epoch in range(1, learning_params['epoch'] + 1):
        model.train()
        
        # Train
        pbar = tqdm(train_data_loader)
        for batch_idx, batch in enumerate(pbar):
            mel_spectrogram_normalised_log_scale_torch, onset_rolls, offset_rolls = batch
        
            x = mel_spectrogram_normalised_log_scale_torch.to(args['device'])
            x = x[:, None, :, :]
            tgt = (onset_rolls.to(args['device']), offset_rolls.to(args['device']))
            out = model(x)
            losses = loss_func.get_loss(out, tgt)
            loss = losses[0]
            metric.update(out, tgt, losses)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_description('Epoch {}, Loss: {:.4f}'.format(epoch, loss.item()))
        metric_train = metric.get_value()

        # Validation
        model.eval()
        with torch.no_grad():
            for batch_idx, batch in enumerate(test_data_loader):
                mel_spectrogram_normalised_log_scale_torch, onset_rolls, offset_rolls = batch

                x = mel_spectrogram_normalised_log_scale_torch.to(args['device'])
                x = x[:, None, :, :]
                tgt = (onset_rolls.to(args['device']), offset_rolls.to(args['device']))
                out = model(x)
                metric.update(out, tgt)
        metric_test = metric.get_value()

        # Logging
        print('[Epoch {:02d}], Train Loss: {:.5f}, Valid Loss {:.5f}, Time {:.2f}s'.format(
            epoch, metric_train['loss'], metric_test['loss'], time.time() - start_time,
        ))
        print('Split Train Loss, F1 Score: Onset Loss {:.4f} | Onset F1 Score {:.4f} | Offset Loss {:.4f} | Offset F1 Score {:.4f}'.format(
            metric_train['onset_loss'],
            metric_train['offset_loss'],
            metric_train['onset_f1'],
            metric_train['offset_f1']
        ))
        print('Split Test Loss, F1 Score: Onset Loss {:.4f} | Onset F1 Score {:.4f} | Offset Loss {:.4f} | Offset F1 Score {:.4f}'.format(
            metric_test['onset_loss'],
            metric_test['offset_loss'],
            metric_test['onset_f1'],
            metric_test['offset_f1']
        ))

        # Save the best model
        if metric_test['loss'] < min_valid_loss:
            min_valid_loss = metric_test['loss']
            best_model_id = epoch

            save_dict = model.state_dict()
            target_model_path = save_model_dir + '/best_model.pth'
            torch.save(save_dict, target_model_path)

        if abs(metric_test['loss'] - prev_loss) < threshold:
            break

        prev_loss = metric_test['loss']

    print('Training done in {:.1f} minutes.'.format((time.time() - start_time) / 60))
    return best_model_id

In [3]:
#For the synthesized Michigan dataset
pipelineOptions = {}
pipelineOptions['frame_per_sec'] = 50
pipelineOptions['window_length'] = 2048
pipelineOptions['n_fft'] = 2048
pipelineOptions['n_mels'] = 256

syn_train_ds, syn_test_ds, syn_data_loader_train, syn_data_loader_test = get_data_loader_synthesized_michigan(args=Hparams_synthesized_michigan.args, pipelineOptions=pipelineOptions)

In [4]:
from tqdm import tqdm

# data loading
for data in tqdm(syn_data_loader_train):
    mel_spectrogram_normalised_log_scale_torch, onset_rolls, offset_rolls = data
    print(f"(Batch, feature)")
    print(f"Spectrogram: {mel_spectrogram_normalised_log_scale_torch.shape} {type(mel_spectrogram_normalised_log_scale_torch)}")
    print(f"Onset roll: {onset_rolls.shape} {type(onset_rolls)}")
    print(f"Offset roll: {offset_rolls.shape} {type(offset_rolls)}")
    break

  0%|          | 0/25 [00:00<?, ?it/s]

(Batch, feature)
Spectrogram: torch.Size([32, 256, 501]) <class 'torch.Tensor'>
Onset roll: torch.Size([32, 501]) <class 'torch.Tensor'>
Offset roll: torch.Size([32, 501]) <class 'torch.Tensor'>


  0%|          | 0/25 [00:26<?, ?it/s]


In [6]:
# Set learning params
learning_params = {
    'epoch': 10,
    'lr': 1e-3,
}

model = ToneSegment_Base(input_shape=(1, 256, 501))
fit_segmentation(model, syn_data_loader_train, syn_data_loader_test, args=Hparams_synthesized_michigan.args, learning_params=learning_params)

Start training...


Epoch 1, Loss: 0.1457: 100%|██████████| 25/25 [00:31<00:00,  1.25s/it]


[Epoch 01], Train Loss: 0.34599, Valid Loss 0.30307, Time 54.67s
Split Train Loss, F1 Score: Onset Loss 0.1703 | Onset F1 Score 0.1757 | Offset Loss 0.1519 | Offset F1 Score 0.0012
Split Test Loss, F1 Score: Onset Loss 0.1575 | Onset F1 Score 0.1456 | Offset Loss 0.2422 | Offset F1 Score 0.0000


Epoch 2, Loss: 0.1593: 100%|██████████| 25/25 [00:29<00:00,  1.18s/it]


[Epoch 02], Train Loss: 0.15267, Valid Loss 0.19943, Time 107.11s
Split Train Loss, F1 Score: Onset Loss 0.0710 | Onset F1 Score 0.0816 | Offset Loss 0.2335 | Offset F1 Score 0.0000
Split Test Loss, F1 Score: Onset Loss 0.0793 | Onset F1 Score 0.1201 | Offset Loss 0.2395 | Offset F1 Score 0.0000


...

Epoch 9, Loss: 0.1442: 100%|██████████| 25/25 [00:29<00:00,  1.18s/it]


[Epoch 09], Train Loss: 0.14620, Valid Loss 0.14258, Time 473.60s
Split Train Loss, F1 Score: Onset Loss 0.0696 | Onset F1 Score 0.0766 | Offset Loss 0.2334 | Offset F1 Score 0.0000
Split Test Loss, F1 Score: Onset Loss 0.0676 | Onset F1 Score 0.0749 | Offset Loss 0.2412 | Offset F1 Score 0.0000


Epoch 10, Loss: 0.1422: 100%|██████████| 25/25 [00:29<00:00,  1.17s/it]


[Epoch 10], Train Loss: 0.14674, Valid Loss 0.14411, Time 525.38s
Split Train Loss, F1 Score: Onset Loss 0.0700 | Onset F1 Score 0.0767 | Offset Loss 0.2335 | Offset F1 Score 0.0000
Split Test Loss, F1 Score: Onset Loss 0.0675 | Onset F1 Score 0.0766 | Offset Loss 0.2445 | Offset F1 Score 0.0000
Training done in 8.8 minutes.


8

In [5]:
#For the synthesized Michigan dataset
args = Hparams_synthesized_michigan.args
args['dataset_root'] =  os.path.join(os.getcwd(), 'data_synthesized_large')

pipelineOptions = {}
pipelineOptions['frame_per_sec'] = 50
pipelineOptions['window_length'] = 2048
pipelineOptions['n_fft'] = 2048
pipelineOptions['n_mels'] = 256

syn_train_ds, syn_test_ds, syn_data_loader_train, syn_data_loader_test = get_data_loader_synthesized_michigan(args=args, pipelineOptions=pipelineOptions)

In [6]:
# Set learning params
learning_params = {
    'epoch': 10,
    'lr': 1e-3,
}

model = ToneSegment_Base(input_shape=(1, 256, 501))
fit_segmentation(model, syn_data_loader_train, syn_data_loader_test, args=args, learning_params=learning_params)

Start training...


Epoch 1, Loss: 0.1501: 100%|██████████| 250/250 [02:00<00:00,  2.07it/s]


[Epoch 01], Train Loss: 0.16291, Valid Loss 0.14586, Time 165.96s
Split Train Loss, F1 Score: Onset Loss 0.0783 | Onset F1 Score 0.0847 | Offset Loss 0.2296 | Offset F1 Score 0.0001
Split Test Loss, F1 Score: Onset Loss 0.0691 | Onset F1 Score 0.0768 | Offset Loss 0.2367 | Offset F1 Score 0.0000


Epoch 2, Loss: 0.1459: 100%|██████████| 250/250 [01:51<00:00,  2.24it/s]


[Epoch 02], Train Loss: 0.14500, Valid Loss 0.14825, Time 320.80s
Split Train Loss, F1 Score: Onset Loss 0.0690 | Onset F1 Score 0.0760 | Offset Loss 0.2364 | Offset F1 Score 0.0000
Split Test Loss, F1 Score: Onset Loss 0.0704 | Onset F1 Score 0.0779 | Offset Loss 0.2366 | Offset F1 Score 0.0000


...

Epoch 9, Loss: 0.1379: 100%|██████████| 250/250 [01:55<00:00,  2.16it/s]


[Epoch 09], Train Loss: 0.14354, Valid Loss 0.14757, Time 1413.34s
Split Train Loss, F1 Score: Onset Loss 0.0687 | Onset F1 Score 0.0749 | Offset Loss 0.2364 | Offset F1 Score 0.0001
Split Test Loss, F1 Score: Onset Loss 0.0687 | Onset F1 Score 0.0789 | Offset Loss 0.2367 | Offset F1 Score 0.0000


Epoch 10, Loss: 0.1457: 100%|██████████| 250/250 [01:55<00:00,  2.16it/s]


[Epoch 10], Train Loss: 0.14341, Valid Loss 0.14817, Time 1575.50s
Split Train Loss, F1 Score: Onset Loss 0.0686 | Onset F1 Score 0.0748 | Offset Loss 0.2364 | Offset F1 Score 0.0002
Split Test Loss, F1 Score: Onset Loss 0.0716 | Onset F1 Score 0.0766 | Offset Loss 0.2365 | Offset F1 Score 0.0000
Training done in 26.3 minutes.


8

In [10]:
# Set learning params
learning_params = {
    'epoch': 10,
    'lr': 1e-3,
}

model = ToneSegment_Enhanced(input_shape=(1, 256, 501))
fit_segmentation(model, syn_data_loader_train, syn_data_loader_test, args=args, learning_params=learning_params)

Start training...


Epoch 1, Loss: 0.1467: 100%|██████████| 250/250 [01:56<00:00,  2.15it/s]


[Epoch 01], Train Loss: 0.15631, Valid Loss 0.14532, Time 158.31s
Split Train Loss, F1 Score: Onset Loss 0.0750 | Onset F1 Score 0.0813 | Offset Loss 0.2216 | Offset F1 Score 0.0001
Split Test Loss, F1 Score: Onset Loss 0.0690 | Onset F1 Score 0.0763 | Offset Loss 0.2364 | Offset F1 Score 0.0000


Epoch 2, Loss: 0.1523: 100%|██████████| 250/250 [01:44<00:00,  2.39it/s]


[Epoch 02], Train Loss: 0.14471, Valid Loss 0.14489, Time 304.19s
Split Train Loss, F1 Score: Onset Loss 0.0689 | Onset F1 Score 0.0759 | Offset Loss 0.2364 | Offset F1 Score 0.0000
Split Test Loss, F1 Score: Onset Loss 0.0687 | Onset F1 Score 0.0762 | Offset Loss 0.2366 | Offset F1 Score 0.0000


...

Epoch 9, Loss: 0.1439: 100%|██████████| 250/250 [01:44<00:00,  2.40it/s]


[Epoch 09], Train Loss: 0.14315, Valid Loss 0.18331, Time 1325.60s
Split Train Loss, F1 Score: Onset Loss 0.0687 | Onset F1 Score 0.0745 | Offset Loss 0.2364 | Offset F1 Score 0.0000
Split Test Loss, F1 Score: Onset Loss 0.0781 | Onset F1 Score 0.1052 | Offset Loss 0.2365 | Offset F1 Score 0.0000


Epoch 10, Loss: 0.1420: 100%|██████████| 250/250 [01:51<00:00,  2.24it/s]


[Epoch 10], Train Loss: 0.14306, Valid Loss 0.19601, Time 1477.03s
Split Train Loss, F1 Score: Onset Loss 0.0686 | Onset F1 Score 0.0744 | Offset Loss 0.2364 | Offset F1 Score 0.0000
Split Test Loss, F1 Score: Onset Loss 0.0809 | Onset F1 Score 0.1151 | Offset Loss 0.2365 | Offset F1 Score 0.0000
Training done in 24.6 minutes.


4